In [1]:
!pip install --upgrade pip --disable-pip-version-check \
  torch==1.13.1 torchdata==0.5.1 transformers==4.27.2 datasets==2.11.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 99.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 77.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer,GenerationConfig

##Dialogue Summarization##

In [3]:
#loading the data from DialogSum dataset
hg_dataset_name="knkarthick/dialogsum"
dataset=load_dataset(hg_dataset_name)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/knkarthick___csv/knkarthick--dialogsum-c8fac5d84cd35861/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
#to print a sample
print(dataset['test'][0]['dialogue'])
print('-'.join('' for x in range(200)))
print(dataset['test'][0]['summary'])

#Person1#: Ms. Dawson, I need you to take a dictation for me.
#Person2#: Yes, sir...
#Person1#: This should go out as an intra-office memorandum to all employees by this afternoon. Are you ready?
#Person2#: Yes, sir. Go ahead.
#Person1#: Attention all staff... Effective immediately, all office communications are restricted to email correspondence and official memos. The use of Instant Message programs by employees during working hours is strictly prohibited.
#Person2#: Sir, does this apply to intra-office communications only? Or will it also restrict external communications?
#Person1#: It should apply to all communications, not only in this office between employees, but also any outside communications.
#Person2#: But sir, many employees use Instant Messaging to communicate with their clients.
#Person1#: They will just have to change their communication methods. I don't want any - one using Instant Messaging in this office. It wastes too much time! Now, please continue with the memo. Wh

In [9]:
#load the seq2seq model and tokenizer
model_name='google/flan-t5-base'
model=AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer=AutoTokenizer.from_pretrained(model_name,use_fast=True)

In [10]:
#testing the tokenizer and its working
sentence= "What day is it, Jerry?"

sentence_encoded=tokenizer(sentence,return_tensors='pt')
sentence_decoded=tokenizer.decode(sentence_encoded['input_ids'][0],
                                  skip_special_tokens=True)

print('ENCODED SENTENCE:',sentence_encoded['input_ids'][0])
print("\nDECODED SENTENCE:",sentence_decoded)

ENCODED SENTENCE: tensor([  363,   239,    19,    34,     6, 16637,    58,     1])

DECODED SENTENCE: What day is it, Jerry?


##Summarization without Prompt Engineering##

In [11]:
sample_index=[20,60]
for i,index in enumerate(sample_index):
  dialogue=dataset['test'][index]['dialogue']
  summary=dataset['test'][index]['summary']

  inputs=tokenizer(dialogue,return_tensors='pt')
  output=tokenizer.decode(model.generate(inputs['input_ids'],
                                         max_new_tokens=50,)[0],
                          skip_special_tokens=True)
  print('-'.join('' for x in range(100)))
  print("Example ",i+1)
  print('-'.join('' for x in range(100)))
  print(f"INPUT PROMPT:\n{dialogue}")
  print('-'.join('' for x in range(100)))
  print(f"BASELINE HUMAN SUMMARY:\n{summary}")
  print('-'.join('' for x in range(100)))
  print(f"MODEL GENERATED SUMMARY - WITHOUT PROMPT ENGINEERING:\n{output}\n")

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What's wrong with you? Why are you scratching so much?
#Person2#: I feel itchy! I can't stand it anymore! I think I may be coming down with something. I feel lightheaded and weak.
#Person1#: Let me have a look. Whoa! Get away from me!
#Person2#: What's wrong?
#Person1#: I think you have chicken pox! You are contagious! Get away! Don't breathe on me!
#Person2#: Maybe it's just a rash or an allergy! We can't be sure until I see a doctor.
#Person1#: Well in the meantime you are a biohazard! I didn't get it when I was a kid and I've heard that you can even die if you get it as an adult!
#Person2#: Are you serious? You always blow things out of proportion. In any case, I think I'll go take an oatmeal bath.
-----------------------------------------------------

#Inference with Instruction Prompt

##Zero Shot##

In [18]:
#prompt1
sample_index=[20,60]
for i,index in enumerate(sample_index):
  dialogue=dataset['test'][index]['dialogue']
  summary=dataset['test'][index]['summary']

  prompt=f"""
Give an accurate summary of the following conversation with exact reasons from the plot.

{dialogue}

Summary:
  """
  inputs=tokenizer(prompt,return_tensors='pt')
  output=tokenizer.decode(model.generate(inputs['input_ids'],
                                         max_new_tokens=50,)[0],
                          skip_special_tokens=True)
  print('-'.join('' for x in range(100)))
  print("Example ",i+1)
  print('-'.join('' for x in range(100)))
  print(f"INPUT PROMPT:\n{prompt}")
  print('-'.join('' for x in range(100)))
  print(f"BASELINE HUMAN SUMMARY:\n{summary}")
  print('-'.join('' for x in range(100)))
  print(f"MODEL GENERATED SUMMARY - ZERO SHOT:\n{output}\n")

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Give an accurate summary of the following conversation with exact reasons from the plot.

#Person1#: What's wrong with you? Why are you scratching so much?
#Person2#: I feel itchy! I can't stand it anymore! I think I may be coming down with something. I feel lightheaded and weak.
#Person1#: Let me have a look. Whoa! Get away from me!
#Person2#: What's wrong?
#Person1#: I think you have chicken pox! You are contagious! Get away! Don't breathe on me!
#Person2#: Maybe it's just a rash or an allergy! We can't be sure until I see a doctor.
#Person1#: Well in the meantime you are a biohazard! I didn't get it when I was a kid and I've heard that you can even die if you get it as an adult!
#Person2#: Are you serious? You always blow things out of proportion. In any case, 

In [19]:
#prompt2
sample_index=[20,60]
for i,index in enumerate(sample_index):
  dialogue=dataset['test'][index]['dialogue']
  summary=dataset['test'][index]['summary']

  prompt=f"""
Summarize the following conversation.

{dialogue}

Summary:
  """
  inputs=tokenizer(prompt,return_tensors='pt')
  output=tokenizer.decode(model.generate(inputs['input_ids'],
                                         max_new_tokens=50,)[0],
                          skip_special_tokens=True)
  print('-'.join('' for x in range(100)))
  print("Example ",i+1)
  print('-'.join('' for x in range(100)))
  print(f"INPUT PROMPT:\n{prompt}")
  print('-'.join('' for x in range(100)))
  print(f"BASELINE HUMAN SUMMARY:\n{summary}")
  print('-'.join('' for x in range(100)))
  print(f"MODEL GENERATED SUMMARY - ZERO SHOT:\n{output}\n")

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: What's wrong with you? Why are you scratching so much?
#Person2#: I feel itchy! I can't stand it anymore! I think I may be coming down with something. I feel lightheaded and weak.
#Person1#: Let me have a look. Whoa! Get away from me!
#Person2#: What's wrong?
#Person1#: I think you have chicken pox! You are contagious! Get away! Don't breathe on me!
#Person2#: Maybe it's just a rash or an allergy! We can't be sure until I see a doctor.
#Person1#: Well in the meantime you are a biohazard! I didn't get it when I was a kid and I've heard that you can even die if you get it as an adult!
#Person2#: Are you serious? You always blow things out of proportion. In any case, I think I'll go take an oatmeal bath.

Summary:
  


In [20]:
#prompt template1
sample_index=[20,60]
for i,index in enumerate(sample_index):
  dialogue=dataset['test'][index]['dialogue']
  summary=dataset['test'][index]['summary']

  prompt=f"""
Dialogue:

{dialogue}

What was going on?
  """
  inputs=tokenizer(prompt,return_tensors='pt')
  output=tokenizer.decode(model.generate(inputs['input_ids'],
                                         max_new_tokens=50,)[0],
                          skip_special_tokens=True)
  print('-'.join('' for x in range(100)))
  print("Example ",i+1)
  print('-'.join('' for x in range(100)))
  print(f"INPUT PROMPT:\n{prompt}")
  print('-'.join('' for x in range(100)))
  print(f"BASELINE HUMAN SUMMARY:\n{summary}")
  print('-'.join('' for x in range(100)))
  print(f"MODEL GENERATED SUMMARY - ZERO SHOT:\n{output}\n")

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Dialogue:

#Person1#: What's wrong with you? Why are you scratching so much?
#Person2#: I feel itchy! I can't stand it anymore! I think I may be coming down with something. I feel lightheaded and weak.
#Person1#: Let me have a look. Whoa! Get away from me!
#Person2#: What's wrong?
#Person1#: I think you have chicken pox! You are contagious! Get away! Don't breathe on me!
#Person2#: Maybe it's just a rash or an allergy! We can't be sure until I see a doctor.
#Person1#: Well in the meantime you are a biohazard! I didn't get it when I was a kid and I've heard that you can even die if you get it as an adult!
#Person2#: Are you serious? You always blow things out of proportion. In any case, I think I'll go take an oatmeal bath.

What was going on?
  
------------------

##ONE and FEW SHOT INFERENCE

In [23]:
#One shot
def make_prompt(example_indices_full,example_index_to_summarize):
  prompt=''
  for index in example_indices_full:
    dialogue=dataset['test'][index]['dialogue']
    summary=dataset['test'][index]['summary']

    prompt+= f"""
Dialogue:

{dialogue}

What was going on?
{summary}


  """
  dialogue=dataset['test'][example_index_to_summarize]['dialogue']
  prompt+=f"""
Dialogue:

{dialogue}

What was going on?
  """
  return prompt

In [24]:
example_indices_full=[40]
example_index_to_summarize=200

one_shot_prompt=make_prompt(example_indices_full,example_index_to_summarize)
print(one_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


  
Dialogue:

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also 

In [26]:
summary=dataset['test'][example_index_to_summarize]['summary']
inputs=tokenizer(one_shot_prompt,return_tensors='pt')
output=tokenizer.decode(model.generate(inputs['input_ids'],
                                       max_new_tokens=50,)[0],
                        skip_special_tokens=True)

print('-'.join('' for x in range(100)))
print(f"BASELINE HUMAN SUMMARY:\n{summary}\n")
print('-'.join('' for x in range(100)))
print(f"MODEL GENERATED SUMMARY - ONE SHOT:\n{output}")

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATED SUMMARY - ONE SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


##FEW SHOT

In [27]:
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Dialogue:

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


  
Dialogue:

#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#: All set

In [29]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

dash_line='-'.join('' for x in range(100))

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.


#Generative Configuration Parameters for Inference

In [31]:
# generation_config = GenerationConfig(max_new_tokens=50)
# generation_config = GenerationConfig(max_new_tokens=10)
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
#Person1 recommends upgrading the system and hardware.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

